<center>
    <h1><u>AML Infections Data Analysis</u></h1>
</center>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import plot, show, savefig, xlim, figure, \
                ylim, legend, boxplot, setp, axes

#run once
#pip install openpyxl
#pip install lifelines

admissions = pd.read_excel('aml_data_8.16.23.xlsx')
#admissions.head()
len(admissions)

In [ ]:
neg_ads = admissions[admissions['infection_present'] == 0]
print('There are '+str(len(neg_ads))+' infection-negative admissions.')

In [ ]:
len(admissions.columns)

# Survival Analysis: Kaplan Meier Curve

In [ ]:
from lifelines import KaplanMeierFitter
# infected vs not infected
kmf = KaplanMeierFitter()
kmc_ads = admissions.drop_duplicates(subset='MRN',keep='last')
X = kmc_ads['survival_months']
Y = kmc_ads['alive_dead']
score_group = kmc_ads['infection_present'] == 1
ax = plt.subplot(111)
kmf.fit(X[~score_group], event_observed = Y[~score_group], label = 'No Infection')
kmf.plot(ax = ax)
kmf.fit(X[score_group], event_observed = Y[score_group], label = 'Infection')
kmf.plot(ax = ax)
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
plt.rc('font', **font)
#plt.title("Kaplan Meier Estimates by Infection/No Infection")
plt.xlabel("Month After AML diagnosis")
plt.ylabel("Probability of Survival")
plt.savefig('KM_infections.png',bbox_inches='tight')

In [ ]:
print(len(Y)) # num patients
print(sum(Y)) # num deaths
print(sum(Y)/len(Y)) # death ratio

In [ ]:
# logrank test
from lifelines.statistics import logrank_test
results=logrank_test(X[~score_group],X[score_group],event_observed_A=Y[~score_group], event_observed_B=Y[score_group])
results.print_summary()

In [ ]:
# race (white vs. other)
kmf = KaplanMeierFitter()
kmc_ads = admissions.drop_duplicates(subset='MRN',keep='last')
X = kmc_ads['survival_months']
Y = kmc_ads['alive_dead']
score_group = kmc_ads['white_caucasian'] == 1
ax = plt.subplot(111)
kmf.fit(X[score_group], event_observed = Y[score_group], label = 'White/Caucasian')
kmf.plot(ax = ax)
kmf.fit(X[~score_group], event_observed = Y[~score_group], label = 'Other')
kmf.plot(ax = ax)
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
plt.rc('font', **font)
#plt.title("Kaplan Meier Estimates by Race")
plt.xlabel("Month After AML diagnosis")
plt.ylabel("Probability of Survival")
plt.savefig('KM_race.png',bbox_inches='tight')

In [ ]:
results=logrank_test(X[score_group],X[~score_group],event_observed_A=Y[score_group], event_observed_B=Y[~score_group])
results.print_summary()

# Machine Learning Analysis

## Explanation

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, show, savefig, xlim, figure, \
                ylim, legend, boxplot, setp, axes
%matplotlib inline
import statsmodels.api as sm

admissions = pd.read_excel('aml_data_8.16.23.xlsx')
admissions.dropna(subset=['cytarabine'],inplace=True)
ads_cv = pd.get_dummies(admissions['CV_line'], drop_first = True)
cvs = ads_cv.columns
admissions = pd.concat([admissions, ads_cv], axis=1)
admissions['male'] = admissions['gender'].map({'M':1, 'F':0})

print('There are '+str(len(admissions))+' total admissions.')
neg_ads = admissions[admissions['infection_present'] == 0]
print('There are '+str(len(neg_ads))+' infection-negative admissions.')

### Univariate Analysis & Logistic Regression

In [ ]:
alpha = .05

In [ ]:
pos_ads = admissions[admissions['infection_present'] == 1]
print('There are '+str(len(pos_ads))+' infection-positive admissions.')
print('There are '+str(len(pos_ads.MRN.unique()))+' patients that were infected.')
#pos_ads.head()
neg_ads = admissions[admissions['infection_present'] == 0]
print('There are '+str(len(neg_ads))+' infection-negative admissions.')
#neg_ads.head()

#### Cytarabine

In [ ]:
admissions['Cytarabine_g/m2/day'] = admissions['Cytarabine mg/m2/day']*0.001
#admissions['Cytarabine_g/m2/day'] = admissions['Cytarabine_g/m2/day'].replace(np.nan,0)
featureset = ['age','male','white_caucasian','Cytarabine_g/m2/day','infection_present']
ads_logreg = admissions[featureset]
target = ads_logreg['infection_present']
features = ads_logreg.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

In [ ]:
featureset = ['age','male','white_caucasian','cyt_2000','infection_present']
ads_logreg = admissions[featureset]
target = ads_logreg['infection_present']
features = ads_logreg.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

In [ ]:
featureset = ['age','male','white_caucasian','cytarabine','infection_present']
ads_logreg = admissions[featureset]
target = ads_logreg['infection_present']
features = ads_logreg.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

In [ ]:
admissions['Cytarabine mg/m2'] = admissions['Cytarabine mg/m2'].replace(np.nan,0)
featureset = ['age','male','white_caucasian','Cytarabine mg/m2','infection_present']
ads_logreg = admissions[featureset]
target = ads_logreg['infection_present']
features = ads_logreg.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

In [ ]:
# scaling cytarabine (and age)
featureset = ['age','male','white_caucasian','Cytarabine mg/m2/day','infection_present']
ads_logreg = admissions[featureset]
ads_logreg_norm = (ads_logreg-ads_logreg.min())/(ads_logreg.max()-ads_logreg.min())
target = ads_logreg_norm['infection_present']
features = ads_logreg_norm.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

#### Levofloxacin

In [ ]:
featureset = ['age','male','white_caucasian','levo','infection_present']
ads_logreg = admissions[featureset]
target = ads_logreg['infection_present']
features = ads_logreg.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

#### Vancomycin - not reporting anymore

In [ ]:
featureset = ['age','male','white_caucasian','vanco','infection_present']
ads_logreg = admissions[featureset]
target = ads_logreg['infection_present']
features = ads_logreg.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

#### Logistic Regression with  Treatments and Controls (age,sex,race)

In [ ]:
admissions = pd.read_excel('aml_data_8.16.23.xlsx')
admissions.dropna(subset=['Cytarabine mg/m2/day'],inplace=True)
admissions['male'] = admissions['gender'].map({'M':1, 'F':0})

admissions['Cytarabine_g/m2/day'] = admissions['Cytarabine mg/m2/day']*0.001
#admissions['Cytarabine_g/m2/day'] = admissions['Cytarabine_g/m2/day'].replace(np.nan,0)
featureset = ['age','male','white_caucasian','Cytarabine_g/m2/day','levo','infection_present']
ads_logreg = admissions[featureset]
target = ads_logreg['infection_present']
features = ads_logreg.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

In [ ]:
admissions = pd.read_excel('aml_data_8.16.23.xlsx')
admissions.dropna(subset=['Cytarabine mg/m2/day'],inplace=True)
admissions['male'] = admissions['gender'].map({'M':1, 'F':0})

featureset = ['age','male','white_caucasian','cyt_2000','levo','infection_present']
ads_logreg = admissions[featureset]
target = ads_logreg['infection_present']
features = ads_logreg.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

In [ ]:
admissions = pd.read_excel('aml_data_8.16.23.xlsx')
admissions.dropna(subset=['Cytarabine mg/m2/day'],inplace=True)
ads_cv = pd.get_dummies(admissions['CV_line'], drop_first = True)
cvs = ads_cv.columns
admissions = pd.concat([admissions, ads_cv], axis=1)
admissions['male'] = admissions['gender'].map({'M':1, 'F':0})

featureset = ['age','male','white_caucasian','cytarabine','levo','infection_present']
ads_logreg = admissions[featureset]
target = ads_logreg['infection_present']
features = ads_logreg.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

In [ ]:
admissions = pd.read_excel('aml_data_8.16.23.xlsx')
admissions.dropna(subset=['Cytarabine mg/m2/day'],inplace=True)
ads_cv = pd.get_dummies(admissions['CV_line'], drop_first = True)
cvs = ads_cv.columns
admissions = pd.concat([admissions, ads_cv], axis=1)
admissions['male'] = admissions['gender'].map({'M':1, 'F':0})

featureset = ['age','male','white_caucasian','Cytarabine mg/m2/day','levo','infection_present']
ads_logreg = admissions[featureset]
ads_logreg_norm = (ads_logreg-ads_logreg.min())/(ads_logreg.max()-ads_logreg.min())
target = ads_logreg_norm['infection_present']
features = ads_logreg_norm.drop(['infection_present'],axis=1)
model = sm.Logit(target, sm.add_constant(features))
result = model.fit()
result.summary()

## Prediction

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, show, savefig, xlim, figure, \
                ylim, legend, boxplot, setp, axes
%matplotlib inline
import statsmodels.api as sm

admissions = pd.read_excel('aml_data_8.16.23.xlsx')
admissions.dropna(subset=['Cytarabine mg/m2/day'],inplace=True)
ads_cv = pd.get_dummies(admissions['CV_line'], drop_first = True)
cvs = ads_cv.columns
admissions = pd.concat([admissions, ads_cv], axis=1)
admissions['male'] = admissions['gender'].map({'M':1, 'F':0})

admissions.first_bmi_kg_m2 = admissions.first_bmi_kg_m2.replace(np.nan,admissions.first_bmi_kg_m2.median())
admissions.dropna(subset=['lowest_neutrophil'],inplace=True)
admissions.dropna(subset=['lowest_platelet'],inplace=True)
admissions.reset_index(inplace=True)
print('There are '+str(len(admissions))+' total admissions.')
neg_ads = admissions[admissions['infection_present'] == 0]
print('There are '+str(len(neg_ads))+' infection-negative admissions.')

There are 426 total admissions.
There are 340 infection-negative admissions.


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve

featureset = ['age',
        'white_caucasian','first_bmi_kg_m2','lowest_neutrophil','lowest_platelet',
        'max_temp_38.5','Cytarabine mg/m2/day','levo',
        'Other', 'Port',
        'male','infection_present']
ads_CART = admissions[featureset]
X = ads_CART.drop(['infection_present'],axis=1)
y = ads_CART['infection_present']

cart = DecisionTreeClassifier()
logreg = LogisticRegression(solver='liblinear')
knn = KNeighborsClassifier()
rf = RandomForestClassifier()

cart_params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}
logreg_params = {'C':[0.0001,0.001,0.01,0.1,1,10,100,250,500,750,1000],'penalty':['l1','l2']}
knn_params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10],'weights':['uniform','distance']}
rf_params = {'n_estimators':[10, 100, 250, 500, 1000],'max_depth':[1,2,5,8,10]} # if slow, only do 1000 n_est

### LOO (Leave One Out) Cross Validation *(<6 hrs)*

In [ ]:
loo = LeaveOneOut()
cart_preds = []
logreg_preds = []
knn_preds = []
rf_preds = []
feat_imp = []
y_tests = []
logreg_betas = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_tests.append(y_test)

    cartGS = GridSearchCV(cart, cart_params, cv=3, scoring='roc_auc')
    cartGS.fit(X_train,y_train)
    cart_preds.append(cartGS.best_estimator_.predict_proba(X_test).T[1])

    logregGS = GridSearchCV(logreg, logreg_params, cv=3, scoring='roc_auc')
    logregGS.fit(X_train,y_train)
    logreg_preds.append(logregGS.best_estimator_.predict_proba(X_test).T[1])
    #Store betas
    logreg_betas.append(logregGS.best_estimator_.coef_)

    knnGS = GridSearchCV(knn, knn_params, cv=3, scoring='roc_auc')
    knnGS.fit(X_train,y_train)
    knn_preds.append(knnGS.best_estimator_.predict_proba(X_test).T[1])

    rfGS = GridSearchCV(rf, rf_params, cv=3, scoring='roc_auc')
    rfGS.fit(X_train,y_train)
    rf_preds.append(rfGS.best_estimator_.predict_proba(X_test).T[1])
    #Store feature importances
    feat_imp.append(rfGS.best_estimator_.feature_importances_)

In [ ]:

# Saving our lists/findings
import pickle
with open('cart_preds1.pickle', 'wb') as handle:
    pickle.dump(cart_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('logreg_preds1.pickle', 'wb') as handle:
    pickle.dump(logreg_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('knn_preds1.pickle', 'wb') as handle:
    pickle.dump(knn_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('rf_preds1.pickle', 'wb') as handle:
    pickle.dump(rf_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('feat_imp1.pickle', 'wb') as handle:
    pickle.dump(feat_imp, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('y_tests1.pickle', 'wb') as handle:
    pickle.dump(y_tests, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('logreg_betas1.pickle', 'wb') as handle:
    pickle.dump(logreg_betas, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Retrieve lists/findings
import pickle
with open('cart_preds5.pickle', 'rb') as handle:
    cart_preds = pickle.load(handle)
with open('logreg_preds5.pickle', 'rb') as handle:
    logreg_preds = pickle.load(handle)
with open('knn_preds5.pickle', 'rb') as handle:
    knn_preds = pickle.load(handle)
with open('rf_preds5.pickle', 'rb') as handle:
    rf_preds = pickle.load(handle)
with open('feat_imp5.pickle', 'rb') as handle:
    feat_imp = pickle.load(handle)
with open('y_tests5.pickle', 'rb') as handle:
    y_tests = pickle.load(handle)
with open('logreg_betas5.pickle', 'rb') as handle:
    logreg_betas = pickle.load(handle)

In [ ]:
#Build Final ROC plot
from sklearn.metrics import roc_auc_score, roc_curve
fpr, tpr, thresh = roc_curve(y_tests, cart_preds)
auc= roc_auc_score(y_tests, cart_preds)

fpr1, tpr1, thresh1 = roc_curve(y_tests, logreg_preds)
auc1= roc_auc_score(y_tests, logreg_preds)

fpr2, tpr2, thresh2 = roc_curve(y_tests, knn_preds)
auc2= roc_auc_score(y_tests, knn_preds)

fpr3, tpr3, thresh3 = roc_curve(y_tests, rf_preds)
auc3= roc_auc_score(y_tests, rf_preds)

fig = plt.figure(figsize=(10, 6))
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
plt.rc('font', **font)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(0.62,0.93,'o', label='Fever')
plt.plot(0.84,0.96,'o', label='Neutropenia')
plt.plot(fpr, tpr, label='CART (AUC = %0.3f)' % auc)
plt.plot(fpr1, tpr1, label='Logistic regression (AUC = %0.3f)' % auc1)
plt.plot(fpr2, tpr2, label='KNN (AUC = %0.3f)' % auc2)
plt.plot(fpr3, tpr3, label='Random Forest (AUC = %0.3f)' % auc3)
#plt.plot(fpr4, tpr4, label='ANC Log Reg (AUC = %0.3f)' % auc4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right');
plt.show()
fig.savefig('prediction_models5.png',bbox_inches='tight') #cyt no thresh w/hemo & mono

Logistic model performs similarly to the simple Fever model. 

In [ ]:
# looking for fpr3 where tpr3 == 0.95, look at left-most point for RF
#tpr3[131] #0.93617
#tpr3[132] #0.9574
#fpr3[131] #0.693
fpr3[132] #0.66, same as fpr1[131]

LR model reduces FPR from 0.76 to 0.69 compared to the Neutropenia model. This is a 9.2% reduction.

Logistic Regression, Random Forest perform similarly to simple Fever

In [ ]:
# looking for fpr1 where tpr1 == 0.94, look at left-most point for LR
#tpr1[111] # 0.93617
fpr1[111]

1/23 The top 5 most important features in the random forest model:
1. max_temp_38.5
2. lowest_neutrophil
3. lowest_platelet
4. age
5. first_bmi_kg_m2

In [ ]:
sum(feat_imp)/len(feat_imp)

In [ ]:
featureset = ['age','white_caucasian','first_bmi_kg_m2','lowest_neutrophil','lowest_platelet',
        'max_temp_38.5','Cytarabine mg/m2/day','levo','Other', 'Port',
        'male',
        'infection_present']

### Build Baseline Fever, Baseline ANC, Log Reg ANC, and compare to RF

#### Using our best model that has cyt_2000 and hemoglobin and monocytes (pickles #6)

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, show, savefig, xlim, figure, \
                ylim, legend, boxplot, setp, axes
%matplotlib inline
import statsmodels.api as sm

admissions = pd.read_excel('aml_data_8.16.23.xlsx')
admissions.dropna(subset=['Cytarabine mg/m2/day'],inplace=True)
ads_cv = pd.get_dummies(admissions['CV_line'], drop_first = True)
cvs = ads_cv.columns
admissions = pd.concat([admissions, ads_cv], axis=1)
admissions['male'] = admissions['gender'].map({'M':1, 'F':0})

admissions.first_bmi_kg_m2 = admissions.first_bmi_kg_m2.replace(np.nan,admissions.first_bmi_kg_m2.median())
admissions.dropna(subset=['lowest_neutrophil'],inplace=True)
admissions.dropna(subset=['lowest_platelet'],inplace=True)
admissions.dropna(subset=['lowest_hemoglobin'],inplace=True)
admissions.dropna(subset=['lowest_monocytes'],inplace=True)
admissions.reset_index(inplace=True)
print('There are '+str(len(admissions))+' total admissions.')
neg_ads = admissions[admissions['infection_present'] == 0]
print('There are '+str(len(neg_ads))+' infection-negative admissions.')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve


featureset = ['age',
        'white_caucasian','first_bmi_kg_m2','lowest_neutrophil','lowest_platelet',
        'max_temp_38.5','cyt_2000','levo',
        'lowest_hemoglobin','lowest_monocytes',
        'Other', 'Port',
        'male','infection_present']
ads_CART = admissions[featureset]
X = ads_CART.drop(['infection_present'],axis=1)
y = ads_CART['infection_present']

# >6 hrs to run
cart = DecisionTreeClassifier()
logreg = LogisticRegression(solver='liblinear')
knn = KNeighborsClassifier()
rf = RandomForestClassifier()

cart_params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}
logreg_params = {'C':[0.0001,0.001,0.01,0.1,1,10,100,250,500,750,1000],'penalty':['l1','l2']}
knn_params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10],'weights':['uniform','distance']}
rf_params = {'n_estimators':[10, 100, 250, 500, 1000],'max_depth':[1,2,5,8,10]} # if slow, only do 1000 n_est

flogreg_score = []
cart_score = []
logreg_score = []
knn_score = []
rf_score = []

In [3]:
import pickle
with open('cart_preds6.pickle', 'rb') as handle:
    cart_preds = pickle.load(handle)
with open('logreg_preds6.pickle', 'rb') as handle:
    logreg_preds = pickle.load(handle)
with open('knn_preds6.pickle', 'rb') as handle:
    knn_preds = pickle.load(handle)
with open('rf_preds6.pickle', 'rb') as handle:
    rf_preds = pickle.load(handle)
with open('feat_imp6.pickle', 'rb') as handle:
    feat_imp = pickle.load(handle)
with open('y_tests6.pickle', 'rb') as handle:
    y_tests = pickle.load(handle)
# with open('logreg_betas6.pickle', 'rb') as handle:
#     logreg_betas = pickle.load(handle)

In [ ]:
# Baseline fever model
# (0, recall) is specificity/TNR. FPR = 1-0.38 = 0.62
# (1, recall) is sensitivity/TPR = 0.93
from sklearn.metrics import classification_report
fevers = admissions['max_temp_38.5']
infections = admissions['infection_present']
print(classification_report(infections,fevers))

In [ ]:
# Baseline ANC model
anc = admissions['neutropenia']
infections = admissions['infection_present']
print(classification_report(infections,anc))

In [ ]:
fpr, tpr, thresh = roc_curve(y_tests, cart_preds)
auc= roc_auc_score(y_tests, cart_preds)

fpr1, tpr1, thresh1 = roc_curve(y_tests, logreg_preds)
auc1= roc_auc_score(y_tests, logreg_preds)

fpr2, tpr2, thresh2 = roc_curve(y_tests, knn_preds)
auc2= roc_auc_score(y_tests, knn_preds)

fpr3, tpr3, thresh3 = roc_curve(y_tests, rf_preds)
auc3= roc_auc_score(y_tests, rf_preds)

fig = plt.figure(figsize=(10, 6))
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
plt.rc('font', **font)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(0.62,0.93,'o', label='Fever')
plt.plot(0.84,0.96,'o', label='Neutropenia')
plt.plot(fpr, tpr, label='CART (AUC = %0.3f)' % auc)
plt.plot(fpr1, tpr1, label='Logistic regression (AUC = %0.3f)' % auc1)
plt.plot(fpr2, tpr2, label='KNN (AUC = %0.3f)' % auc2)
plt.plot(fpr3, tpr3, label='Random Forest (AUC = %0.3f)' % auc3)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right');
plt.show()
#fig.savefig('prediction_models6.png',bbox_inches='tight')

In [ ]:
# looking for fpr3 where tpr3 == 0.93, look at left-most point for RF
#tpr3[113] #0.9176
#tpr3[114] #0.9412
#fpr3[113] #0.6893
fpr3[114] #0.6893, same as fpr1[113]

In [ ]:
# looking for fpr1 where tpr1 == 0.93, look at left-most point for LR
#tpr1[107] #
fpr1[108]

In [ ]:
# comparing LR to Neutropenia TPR=0.96
tpr1[113] #0.965
fpr1[113]

In [4]:
sum(feat_imp)/len(feat_imp)

array([0.07222225, 0.0124984 , 0.05511358, 0.17220583, 0.0927168 ,
       0.21049094, 0.11793829, 0.05036775, 0.04097311, 0.15306527,
       0.00672387, 0.00321798, 0.01246592])

In [ ]:
featureset = ['age','white_caucasian','first_bmi_kg_m2','lowest_neutrophil','lowest_platelet',
        'max_temp_38.5','cyt_2000','levo','Other', 'Port',
        'lowest_hemoglobin','lowest_monocytes','male',
        'infection_present']

In [ ]:
#Find proportion positive
from sklearn.metrics import confusion_matrix
cMatrix = confusion_matrix(y_true = infections, y_pred = fevers)
fig = plt.figure(figsize=(10, 6))
plt.imshow(cMatrix, cmap=plt.cm.Blues)
plt.text(0, 0, '{}'.format(cMatrix[0, 0]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(0, 1, '{}'.format(cMatrix[1, 0]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(1, 0, '{}'.format(cMatrix[0, 1]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(1, 1, '{}'.format(cMatrix[1, 1]), horizontalalignment='center',fontsize = 'xx-large')
tick_marks = [0,1]
labels = ['No infection', 'Infection']
plt.xticks(tick_marks, labels, rotation=90,fontsize = 'x-large')
plt.ylim([-0.5,1.5])
plt.yticks(tick_marks, labels,fontsize = 'x-large')
plt.ylabel('True label',fontsize = 'xx-large')
plt.xlabel('Predicted label',fontsize = 'xx-large')
plt.show()

In [ ]:
#Find proportion positive
from sklearn.metrics import confusion_matrix
cMatrix = confusion_matrix(y_true = infections, y_pred = anc)
fig = plt.figure(figsize=(10, 6))
plt.imshow(cMatrix, cmap=plt.cm.Blues)
plt.text(0, 0, '{}'.format(cMatrix[0, 0]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(0, 1, '{}'.format(cMatrix[1, 0]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(1, 0, '{}'.format(cMatrix[0, 1]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(1, 1, '{}'.format(cMatrix[1, 1]), horizontalalignment='center',fontsize = 'xx-large')
tick_marks = [0,1]
labels = ['No infection', 'Infection']
plt.xticks(tick_marks, labels, rotation=90,fontsize = 'x-large')
plt.ylim([-0.5,1.5])
plt.yticks(tick_marks, labels,fontsize = 'x-large')
plt.ylabel('True label',fontsize = 'xx-large')
plt.xlabel('Predicted label',fontsize = 'xx-large')
plt.show()

In [ ]:
rf_preds_class = np.concatenate(rf_preds).ravel()

In [ ]:
# Comparing TPR of Fever model to same of RF
rfrounds = []
for i in rf_preds_class:
    if i > thresh3[next(x[0] for x in enumerate(tpr3) if x[1] > 0.93)]:
        rfrounds.append(1)
    else:
        rfrounds.append(0)
print(thresh3[next(x[0] for x in enumerate(tpr3) if x[1] > 0.93)])

In [ ]:
from sklearn.metrics import classification_report
rf_preds_f = pd.cut(rf_preds_class,bins=[-1,0.1643888587661629,1],right=True,labels=[0,1])
#infections = ads_CART['infection_present']
print(classification_report(y_tests,rf_preds_f))

In [ ]:
#Positive Proportion calc
from sklearn.metrics import confusion_matrix
cMatrix = confusion_matrix(y_true = y_tests, y_pred = rf_preds_f)
fig = plt.figure(figsize=(10, 6))
plt.imshow(cMatrix, cmap=plt.cm.Blues)
plt.text(0, 0, '{}'.format(cMatrix[0, 0]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(0, 1, '{}'.format(cMatrix[1, 0]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(1, 0, '{}'.format(cMatrix[0, 1]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(1, 1, '{}'.format(cMatrix[1, 1]), horizontalalignment='center',fontsize = 'xx-large')
tick_marks = [0,1]
labels = ['No infection', 'Infection']
plt.xticks(tick_marks, labels, rotation=90,fontsize = 'x-large')
plt.ylim([-0.5,1.5])
plt.yticks(tick_marks, labels,fontsize = 'x-large')
plt.ylabel('True label',fontsize = 'xx-large')
plt.xlabel('Predicted label',fontsize = 'xx-large')
plt.show()

In [ ]:
lr_preds_class = np.concatenate(logreg_preds).ravel()

In [ ]:
# Comparing TPR of Fever model to same of LR
lrrounds = []
for i in lr_preds_class:
    if i > thresh1[next(x[0] for x in enumerate(tpr1) if x[1] > 0.93)]:
        lrrounds.append(1)
    else:
        lrrounds.append(0)
print(thresh1[next(x[0] for x in enumerate(tpr1) if x[1] > 0.93)])

In [ ]:
from sklearn.metrics import classification_report
lr_preds_f = pd.cut(lr_preds_class,bins=[-1,0.058274785260887065,1],right=True,labels=[0,1])
#infections = ads_CART['infection_present']
print(classification_report(y_tests,lr_preds_f))

In [ ]:
#Positive Proportion calc
from sklearn.metrics import confusion_matrix
cMatrix = confusion_matrix(y_true = y_tests, y_pred = lr_preds_f)
fig = plt.figure(figsize=(10, 6))
plt.imshow(cMatrix, cmap=plt.cm.Blues)
plt.text(0, 0, '{}'.format(cMatrix[0, 0]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(0, 1, '{}'.format(cMatrix[1, 0]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(1, 0, '{}'.format(cMatrix[0, 1]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(1, 1, '{}'.format(cMatrix[1, 1]), horizontalalignment='center',fontsize = 'xx-large')
tick_marks = [0,1]
labels = ['No infection', 'Infection']
plt.xticks(tick_marks, labels, rotation=90,fontsize = 'x-large')
plt.ylim([-0.5,1.5])
plt.yticks(tick_marks, labels,fontsize = 'x-large')
plt.ylabel('True label',fontsize = 'xx-large')
plt.xlabel('Predicted label',fontsize = 'xx-large')
plt.show()

In [ ]:
# Comparing TPR of ANC model to same of RF
rfrounds = []
for i in rf_preds_class:
    if i > thresh3[next(x[0] for x in enumerate(tpr3) if x[1] > 0.95)]:
        rfrounds.append(1)
    else:
        rfrounds.append(0)
print(thresh3[next(x[0] for x in enumerate(tpr3) if x[1] > 0.95)])

In [ ]:
from sklearn.metrics import classification_report
rf_preds_f = pd.cut(rf_preds_class,bins=[-1,0.13953274313613712,1],right=True,labels=[0,1])
#infections = ads_CART['infection_present']
print(classification_report(y_tests,rf_preds_f))

In [ ]:
#PPV calc check (1, precision)
from sklearn.metrics import confusion_matrix
cMatrix = confusion_matrix(y_true = y_tests, y_pred = rf_preds_f)
fig = plt.figure(figsize=(10, 6))
plt.imshow(cMatrix, cmap=plt.cm.Blues)
plt.text(0, 0, '{}'.format(cMatrix[0, 0]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(0, 1, '{}'.format(cMatrix[1, 0]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(1, 0, '{}'.format(cMatrix[0, 1]), horizontalalignment='center',fontsize = 'xx-large')
plt.text(1, 1, '{}'.format(cMatrix[1, 1]), horizontalalignment='center',fontsize = 'xx-large')
tick_marks = [0,1]
labels = ['No infection', 'Infection']
plt.xticks(tick_marks, labels, rotation=90,fontsize = 'x-large')
plt.ylim([-0.5,1.5])
plt.yticks(tick_marks, labels,fontsize = 'x-large')
plt.ylabel('True label',fontsize = 'xx-large')
plt.xlabel('Predicted label',fontsize = 'xx-large')
plt.show()

*Below model is shown on ROC plot, but not used to compare with other models in "Our Table 5" as it is basically the normal Log Reg model.*

In [ ]:
# Retrieve ANC Log Reg model results
import pickle
with open('anc_logreg_preds.pickle', 'rb') as handle:
    anc_logreg_preds = pickle.load(handle)
with open('anc_y_tests.pickle', 'rb') as handle:
    anc_y_tests = pickle.load(handle)

In [ ]:
# Build Log Reg ANC model <5 min
X = ads_CART['lowest_neutrophil'].values
X = X.reshape(-1, 1)
y = ads_CART['infection_present']
loo = LeaveOneOut()
anc_logreg_preds = []
anc_y_tests = []
anc_logreg_betas = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    anc_y_tests.append(y_test)
    
    logregGS = GridSearchCV(logreg, logreg_params, cv=3, scoring='roc_auc')
    logregGS.fit(X_train,y_train)
    anc_logreg_preds.append(logregGS.best_estimator_.predict_proba(X_test).T[1])
    anc_logreg_betas.append(logregGS.best_estimator_.coef_)
    #print(logregGS.best_estimator_)

In [ ]:
sum(anc_logreg_betas)/len(anc_logreg_betas)

In [ ]:
fpr4, tpr4, thresh4 = roc_curve(anc_y_tests, anc_logreg_preds)
auc4= roc_auc_score(anc_y_tests, anc_logreg_preds)

In [ ]:
import pickle
with open('anc_logreg_preds.pickle', 'wb') as handle:
    pickle.dump(anc_logreg_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('anc_y_tests.pickle', 'wb') as handle:
    pickle.dump(anc_y_tests, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('anc_logreg_betas.pickle', 'wb') as handle:
    pickle.dump(anc_logreg_betas, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Compare to Fever model at same TPR
anc_logreg_class = np.concatenate(anc_logreg_preds).ravel()
anc_rounds = []
for i in anc_logreg_class:
    if i > thresh4[next(x[0] for x in enumerate(tpr4) if x[1] > 0.87)]:
        anc_rounds.append(1)
    else:
        anc_rounds.append(0)
print(thresh4[next(x[0] for x in enumerate(tpr4) if x[1] > 0.87)])

In [ ]:
anc_logreg_preds_f = pd.cut(anc_logreg_class,bins=[-1,0.08487083115242275,1],right=True,labels=[0,1])
infections = admissions['infection_present']
print(classification_report(infections,anc_logreg_preds_f))

In [ ]:
#Build Final ROC plot
fpr, tpr, thresh = roc_curve(y_tests, cart_preds)
auc= roc_auc_score(y_tests, cart_preds)

fpr1, tpr1, thresh1 = roc_curve(y_tests, logreg_preds)
auc1= roc_auc_score(y_tests, logreg_preds)

fpr2, tpr2, thresh2 = roc_curve(y_tests, knn_preds)
auc2= roc_auc_score(y_tests, knn_preds)

fpr3, tpr3, thresh3 = roc_curve(y_tests, rf_preds)
auc3= roc_auc_score(y_tests, rf_preds)

fig = plt.figure(figsize=(10, 6))
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
plt.rc('font', **font)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(0.58,0.94,'o', label='Fever') # baseline fever model
plt.plot(0.76,0.95,'o', label='Neutropenia') # baseline anc model
plt.plot(fpr, tpr, label='CART (AUC = %0.3f)' % auc)
plt.plot(fpr1, tpr1, label='Logistic regression (AUC = %0.3f)' % auc1)
plt.plot(fpr2, tpr2, label='KNN (AUC = %0.3f)' % auc2)
plt.plot(fpr3, tpr3, label='Random Forest (AUC = %0.3f)' % auc3)
#plt.plot(fpr4, tpr4, label='ANC Log Reg (AUC = %0.3f)' % auc4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right');
plt.show()
fig.savefig('prediction_models.png',bbox_inches='tight')

### Cytarabine Dose Low/High/Very High
- 200mg/m2/day x8 or 10 days (low dose)
- 2000mg/m2/day x 4 or 5 days (high dose)
- 6000mg/m2/day x 4 days (very high)

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, show, savefig, xlim, figure, \
                ylim, legend, boxplot, setp, axes
%matplotlib inline
import statsmodels.api as sm

admissions = pd.read_excel('aml_data_8.16.23.xlsx')
admissions.dropna(subset=['cyt_low'],inplace=True)
admissions.reset_index(inplace=True)
ads_cv = pd.get_dummies(admissions['CV_line'], drop_first = True)
cvs = ads_cv.columns
admissions = pd.concat([admissions, ads_cv], axis=1)
admissions['male'] = admissions['gender'].map({'M':1, 'F':0})

admissions.first_bmi_kg_m2 = admissions.first_bmi_kg_m2.replace(np.nan,admissions.first_bmi_kg_m2.median())
admissions.dropna(subset=['lowest_neutrophil'],inplace=True)
admissions.dropna(subset=['lowest_platelet'],inplace=True)
admissions.reset_index(inplace=True)
print('There are '+str(len(admissions))+' total admissions.')
neg_ads = admissions[admissions['infection_present'] == 0]
print('There are '+str(len(neg_ads))+' infection-negative admissions.')

In [ ]:
# ~4 hours
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve

featureset = ['age',
        'white_caucasian','first_bmi_kg_m2','lowest_neutrophil','lowest_platelet',
        'max_temp_38.5','cyt_low','cyt_high','cyt_very_high','levo',
        'Other', 'Port',
        'male','infection_present']
ads_CART = admissions[featureset]
X = ads_CART.drop(['infection_present'],axis=1)
y = ads_CART['infection_present']

cart = DecisionTreeClassifier()
logreg = LogisticRegression(solver='liblinear')
knn = KNeighborsClassifier()
rf = RandomForestClassifier()

cart_params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}
logreg_params = {'C':[0.0001,0.001,0.01,0.1,1,10,100,250,500,750,1000],'penalty':['l1','l2']}
knn_params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10],'weights':['uniform','distance']}
rf_params = {'n_estimators':[10, 100, 250, 500, 1000],'max_depth':[1,2,5,8,10]} # if slow, only do 1000 n_est

loo = LeaveOneOut()
cart_preds = []
logreg_preds = []
knn_preds = []
rf_preds = []
feat_imp = []
y_tests = []
logreg_betas = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_tests.append(y_test)

    cartGS = GridSearchCV(cart, cart_params, cv=3, scoring='roc_auc')
    cartGS.fit(X_train,y_train)
    cart_preds.append(cartGS.best_estimator_.predict_proba(X_test).T[1])

    logregGS = GridSearchCV(logreg, logreg_params, cv=3, scoring='roc_auc')
    logregGS.fit(X_train,y_train)
    logreg_preds.append(logregGS.best_estimator_.predict_proba(X_test).T[1])
    #Store betas
    logreg_betas.append(logregGS.best_estimator_.coef_)

    knnGS = GridSearchCV(knn, knn_params, cv=3, scoring='roc_auc')
    knnGS.fit(X_train,y_train)
    knn_preds.append(knnGS.best_estimator_.predict_proba(X_test).T[1])

    rfGS = GridSearchCV(rf, rf_params, cv=3, scoring='roc_auc')
    rfGS.fit(X_train,y_train)
    rf_preds.append(rfGS.best_estimator_.predict_proba(X_test).T[1])
    #Store feature importances
    feat_imp.append(rfGS.best_estimator_.feature_importances_)

In [ ]:
# Saving our lists/findings
import pickle
with open('cart_preds2.pickle', 'wb') as handle:
    pickle.dump(cart_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('logreg_preds2.pickle', 'wb') as handle:
    pickle.dump(logreg_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('knn_preds2.pickle', 'wb') as handle:
    pickle.dump(knn_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('rf_preds2.pickle', 'wb') as handle:
    pickle.dump(rf_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('feat_imp2.pickle', 'wb') as handle:
    pickle.dump(feat_imp, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('y_tests2.pickle', 'wb') as handle:
    pickle.dump(y_tests, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('logreg_betas2.pickle', 'wb') as handle:
    pickle.dump(logreg_betas, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Retrieve lists/findings
import pickle
with open('cart_preds2.pickle', 'rb') as handle:
    cart_preds = pickle.load(handle)
with open('logreg_preds2.pickle', 'rb') as handle:
    logreg_preds = pickle.load(handle)
with open('knn_preds2.pickle', 'rb') as handle:
    knn_preds = pickle.load(handle)
with open('rf_preds2.pickle', 'rb') as handle:
    rf_preds = pickle.load(handle)
with open('feat_imp2.pickle', 'rb') as handle:
    feat_imp = pickle.load(handle)
with open('y_tests2.pickle', 'rb') as handle:
    y_tests = pickle.load(handle)

In [ ]:
#Build Final ROC plot
fpr, tpr, thresh = roc_curve(y_tests, cart_preds)
auc= roc_auc_score(y_tests, cart_preds)

fpr1, tpr1, thresh1 = roc_curve(y_tests, logreg_preds)
auc1= roc_auc_score(y_tests, logreg_preds)

fpr2, tpr2, thresh2 = roc_curve(y_tests, knn_preds)
auc2= roc_auc_score(y_tests, knn_preds)

fpr3, tpr3, thresh3 = roc_curve(y_tests, rf_preds)
auc3= roc_auc_score(y_tests, rf_preds)

fig = plt.figure(figsize=(10, 6))
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
plt.rc('font', **font)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(0.58,0.94,'o', label='Fever') # baseline fever model
plt.plot(0.76,0.95,'o', label='Neutropenia') # baseline anc model
plt.plot(fpr, tpr, label='CART (AUC = %0.3f)' % auc)
plt.plot(fpr1, tpr1, label='Logistic regression (AUC = %0.3f)' % auc1)
plt.plot(fpr2, tpr2, label='KNN (AUC = %0.3f)' % auc2)
plt.plot(fpr3, tpr3, label='Random Forest (AUC = %0.3f)' % auc3)
#plt.plot(fpr4, tpr4, label='ANC Log Reg (AUC = %0.3f)' % auc4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right');
plt.show()
fig.savefig('prediction_models2.png',bbox_inches='tight')

In [ ]:
sum(feat_imp)/len(feat_imp)

In [ ]:
featureset = ['age','white_caucasian','first_bmi_kg_m2','lowest_neutrophil','lowest_platelet',
        'max_temp_38.5','cyt_low','cyt_high','cyt_very_high','levo',
        'Other', 'Port','male',
        'infection_present']

### Cytarabine high dose threshold of 1000 or 2000

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, show, savefig, xlim, figure, \
                ylim, legend, boxplot, setp, axes
%matplotlib inline
import statsmodels.api as sm

admissions = pd.read_excel('aml_data_8.16.23.xlsx')
admissions.dropna(subset=['cyt_1000'],inplace=True)
ads_cv = pd.get_dummies(admissions['CV_line'], drop_first = True)
cvs = ads_cv.columns
admissions = pd.concat([admissions, ads_cv], axis=1)
admissions['male'] = admissions['gender'].map({'M':1, 'F':0})

admissions.first_bmi_kg_m2 = admissions.first_bmi_kg_m2.replace(np.nan,admissions.first_bmi_kg_m2.median())
admissions.dropna(subset=['lowest_neutrophil'],inplace=True)
admissions.dropna(subset=['lowest_platelet'],inplace=True)
admissions.reset_index(inplace=True)
print('There are '+str(len(admissions))+' total admissions.')
neg_ads = admissions[admissions['infection_present'] == 0]
print('There are '+str(len(neg_ads))+' infection-negative admissions.')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve

featureset = ['age',
        'white_caucasian','first_bmi_kg_m2','lowest_neutrophil','lowest_platelet',
        'max_temp_38.5','cyt_2000','levo',
        'Other', 'Port',
        'male','infection_present']
ads_CART = admissions[featureset]
X = ads_CART.drop(['infection_present'],axis=1)
y = ads_CART['infection_present']

# >6 hrs to run
cart = DecisionTreeClassifier()
logreg = LogisticRegression(solver='liblinear')
knn = KNeighborsClassifier()
rf = RandomForestClassifier()

cart_params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}
logreg_params = {'C':[0.0001,0.001,0.01,0.1,1,10,100,250,500,750,1000],'penalty':['l1','l2']}
knn_params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10],'weights':['uniform','distance']}
rf_params = {'n_estimators':[10, 100, 250, 500, 1000],'max_depth':[1,2,5,8,10]} # if slow, only do 1000 n_est

flogreg_score = []
cart_score = []
logreg_score = []
knn_score = []
rf_score = []

loo = LeaveOneOut()
cart_preds = []
logreg_preds = []
knn_preds = []
rf_preds = []
feat_imp = []
y_tests = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_tests.append(y_test)

    cartGS = GridSearchCV(cart, cart_params, cv=3, scoring='roc_auc')
    cartGS.fit(X_train,y_train)
    cart_preds.append(cartGS.best_estimator_.predict_proba(X_test).T[1])

    logregGS = GridSearchCV(logreg, logreg_params, cv=3, scoring='roc_auc')
    logregGS.fit(X_train,y_train)
    logreg_preds.append(logregGS.best_estimator_.predict_proba(X_test).T[1])
    #Store betas
    #logreg_betas.append(logregGS.best_estimator_.coef_)

    knnGS = GridSearchCV(knn, knn_params, cv=3, scoring='roc_auc')
    knnGS.fit(X_train,y_train)
    knn_preds.append(knnGS.best_estimator_.predict_proba(X_test).T[1])

    rfGS = GridSearchCV(rf, rf_params, cv=3, scoring='roc_auc')
    rfGS.fit(X_train,y_train)
    rf_preds.append(rfGS.best_estimator_.predict_proba(X_test).T[1])
    #Store feature importances
    feat_imp.append(rfGS.best_estimator_.feature_importances_)

In [ ]:
# Saving our lists/findings
import pickle
with open('cart_preds4.pickle', 'wb') as handle:
    pickle.dump(cart_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('logreg_preds4.pickle', 'wb') as handle:
    pickle.dump(logreg_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('knn_preds4.pickle', 'wb') as handle:
    pickle.dump(knn_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('rf_preds4.pickle', 'wb') as handle:
    pickle.dump(rf_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('feat_imp4.pickle', 'wb') as handle:
    pickle.dump(feat_imp, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('y_tests4.pickle', 'wb') as handle:
    pickle.dump(y_tests, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Retrieve lists/findings
import pickle
with open('cart_preds4.pickle', 'rb') as handle:
    cart_preds = pickle.load(handle)
with open('logreg_preds4.pickle', 'rb') as handle:
    logreg_preds = pickle.load(handle)
with open('knn_preds4.pickle', 'rb') as handle:
    knn_preds = pickle.load(handle)
with open('rf_preds4.pickle', 'rb') as handle:
    rf_preds = pickle.load(handle)
with open('feat_imp4.pickle', 'rb') as handle:
    feat_imp = pickle.load(handle)
with open('y_tests4.pickle', 'rb') as handle:
    y_tests = pickle.load(handle)

In [ ]:
# Baseline fever model
# (0, recall) is specificity/TNR = 1 - FPR
# (1, recall) is sensitivity/TPR = 0.94
from sklearn.metrics import classification_report
fevers = admissions['max_temp_38.5']
infections = admissions['infection_present']
print(classification_report(infections,fevers))

In [ ]:
neutropenia = admissions['neutropenia']
infections = admissions['infection_present']
print(classification_report(infections,neutropenia))

In [ ]:
#Build Final ROC plot
fpr, tpr, thresh = roc_curve(y_tests, cart_preds)
auc= roc_auc_score(y_tests, cart_preds)

fpr1, tpr1, thresh1 = roc_curve(y_tests, logreg_preds)
auc1= roc_auc_score(y_tests, logreg_preds)

fpr2, tpr2, thresh2 = roc_curve(y_tests, knn_preds)
auc2= roc_auc_score(y_tests, knn_preds)

fpr3, tpr3, thresh3 = roc_curve(y_tests, rf_preds)
auc3= roc_auc_score(y_tests, rf_preds)

fig = plt.figure(figsize=(10, 6))
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
plt.rc('font', **font)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(0.58,0.94,'o', label='Fever') # baseline fever model
plt.plot(0.76,0.95,'o', label='Neutropenia') # baseline anc model
plt.plot(fpr, tpr, label='CART (AUC = %0.3f)' % auc)
plt.plot(fpr1, tpr1, label='Logistic regression (AUC = %0.3f)' % auc1)
plt.plot(fpr2, tpr2, label='KNN (AUC = %0.3f)' % auc2)
plt.plot(fpr3, tpr3, label='Random Forest (AUC = %0.3f)' % auc3)
#plt.plot(fpr4, tpr4, label='ANC Log Reg (AUC = %0.3f)' % auc4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right');
plt.show()
fig.savefig('prediction_models4.png',bbox_inches='tight')

In [ ]:
sum(feat_imp)/len(feat_imp)

In [ ]:
featureset = ['age','white_caucasian','first_bmi_kg_m2','lowest_neutrophil','lowest_platelet',
        'max_temp_38.5','cyt_1000','levo','Other', 'Port',
        'male',
        'infection_present']

### What if included hemoglobin and monocytes?

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, show, savefig, xlim, figure, \
                ylim, legend, boxplot, setp, axes
%matplotlib inline
import statsmodels.api as sm

admissions = pd.read_excel('aml_data_8.16.23.xlsx')
admissions.dropna(subset=['Cytarabine mg/m2/day'],inplace=True)
ads_cv = pd.get_dummies(admissions['CV_line'], drop_first = True)
cvs = ads_cv.columns
admissions = pd.concat([admissions, ads_cv], axis=1)
admissions['male'] = admissions['gender'].map({'M':1, 'F':0})

admissions.first_bmi_kg_m2 = admissions.first_bmi_kg_m2.replace(np.nan,admissions.first_bmi_kg_m2.median())
admissions.dropna(subset=['lowest_neutrophil'],inplace=True)
admissions.dropna(subset=['lowest_platelet'],inplace=True)
admissions.dropna(subset=['lowest_hemoglobin'],inplace=True)
admissions.dropna(subset=['lowest_monocytes'],inplace=True)
admissions.reset_index(inplace=True)
print('There are '+str(len(admissions))+' total admissions.')
neg_ads = admissions[admissions['infection_present'] == 0]
print('There are '+str(len(neg_ads))+' infection-negative admissions.')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve


featureset = ['age',
        'white_caucasian','first_bmi_kg_m2','lowest_neutrophil','lowest_platelet',
        'max_temp_38.5','cyt_2000','levo',
        'lowest_hemoglobin','lowest_monocytes',
        'Other', 'Port',
        'male','infection_present']
ads_CART = admissions[featureset]
X = ads_CART.drop(['infection_present'],axis=1)
y = ads_CART['infection_present']

# >6 hrs to run
cart = DecisionTreeClassifier()
logreg = LogisticRegression(solver='liblinear')
knn = KNeighborsClassifier()
rf = RandomForestClassifier()

cart_params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}
logreg_params = {'C':[0.0001,0.001,0.01,0.1,1,10,100,250,500,750,1000],'penalty':['l1','l2']}
knn_params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10],'weights':['uniform','distance']}
rf_params = {'n_estimators':[10, 100, 250, 500, 1000],'max_depth':[1,2,5,8,10]} # if slow, only do 1000 n_est

flogreg_score = []
cart_score = []
logreg_score = []
knn_score = []
rf_score = []

loo = LeaveOneOut()
cart_preds = []
logreg_preds = []
knn_preds = []
rf_preds = []
feat_imp = []
y_tests = []
logreg_betas = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_tests.append(y_test)

    cartGS = GridSearchCV(cart, cart_params, cv=3, scoring='roc_auc')
    cartGS.fit(X_train,y_train)
    cart_preds.append(cartGS.best_estimator_.predict_proba(X_test).T[1])

    logregGS = GridSearchCV(logreg, logreg_params, cv=3, scoring='roc_auc')
    logregGS.fit(X_train,y_train)
    logreg_preds.append(logregGS.best_estimator_.predict_proba(X_test).T[1])
    #Store betas
    logreg_betas.append(logregGS.best_estimator_.coef_)

    knnGS = GridSearchCV(knn, knn_params, cv=3, scoring='roc_auc')
    knnGS.fit(X_train,y_train)
    knn_preds.append(knnGS.best_estimator_.predict_proba(X_test).T[1])

    rfGS = GridSearchCV(rf, rf_params, cv=3, scoring='roc_auc')
    rfGS.fit(X_train,y_train)
    rf_preds.append(rfGS.best_estimator_.predict_proba(X_test).T[1])
    #Store feature importances
    feat_imp.append(rfGS.best_estimator_.feature_importances_)

In [ ]:
# Saving our lists/findings
import pickle
with open('cart_preds8.pickle', 'wb') as handle:
    pickle.dump(cart_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('logreg_preds8.pickle', 'wb') as handle:
    pickle.dump(logreg_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('knn_preds8.pickle', 'wb') as handle:
    pickle.dump(knn_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('rf_preds8.pickle', 'wb') as handle:
    pickle.dump(rf_preds, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('feat_imp8.pickle', 'wb') as handle:
    pickle.dump(feat_imp, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('y_tests8.pickle', 'wb') as handle:
    pickle.dump(y_tests, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('logreg_betas8.pickle', 'wb') as handle:
    pickle.dump(logreg_betas, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Baseline fever model
# (0, recall) is specificity/TNR = 1 - FPR
# (1, recall) is sensitivity/TPR = 0.94
from sklearn.metrics import classification_report
fevers = admissions['max_temp_38.5']
infections = admissions['infection_present']
print(classification_report(infections,fevers))

In [ ]:
neutropenia = admissions['neutropenia']
infections = admissions['infection_present']
print(classification_report(infections,neutropenia))

In [ ]:
#Build Final ROC plot
fpr, tpr, thresh = roc_curve(y_tests, cart_preds)
auc= roc_auc_score(y_tests, cart_preds)

fpr1, tpr1, thresh1 = roc_curve(y_tests, logreg_preds)
auc1= roc_auc_score(y_tests, logreg_preds)

fpr2, tpr2, thresh2 = roc_curve(y_tests, knn_preds)
auc2= roc_auc_score(y_tests, knn_preds)

fpr3, tpr3, thresh3 = roc_curve(y_tests, rf_preds)
auc3= roc_auc_score(y_tests, rf_preds)

fig = plt.figure(figsize=(10, 6))
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
plt.rc('font', **font)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(0.58,0.94,'o', label='Fever') # baseline fever model
plt.plot(0.76,0.95,'o', label='Neutropenia') # baseline anc model
plt.plot(fpr, tpr, label='CART (AUC = %0.3f)' % auc)
plt.plot(fpr1, tpr1, label='Logistic regression (AUC = %0.3f)' % auc1)
plt.plot(fpr2, tpr2, label='KNN (AUC = %0.3f)' % auc2)
plt.plot(fpr3, tpr3, label='Random Forest (AUC = %0.3f)' % auc3)
#plt.plot(fpr4, tpr4, label='ANC Log Reg (AUC = %0.3f)' % auc4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right');
plt.show()
fig.savefig('prediction_models8.png',bbox_inches='tight')

In [ ]:
# Retrieve lists/findings
import pickle
with open('cart_preds8.pickle', 'rb') as handle:
    cart_preds = pickle.load(handle)
with open('logreg_preds8.pickle', 'rb') as handle:
    logreg_preds = pickle.load(handle)
with open('knn_preds8.pickle', 'rb') as handle:
    knn_preds = pickle.load(handle)
with open('rf_preds8.pickle', 'rb') as handle:
    rf_preds = pickle.load(handle)
with open('feat_imp8.pickle', 'rb') as handle:
    feat_imp = pickle.load(handle)
with open('y_tests8.pickle', 'rb') as handle:
    y_tests = pickle.load(handle)
with open('logreg_betas8.pickle', 'rb') as handle:
    logreg_betas = pickle.load(handle)

In [ ]:
sum(logreg_betas)/len(logreg_betas)

In [ ]:
sum(feat_imp)/len(feat_imp)

In [ ]:
featureset = ['age','white_caucasian','first_bmi_kg_m2','lowest_neutrophil','lowest_platelet',
        'max_temp_38.5','cyt_2000','levo','Other', 'Port',
        'lowest_hemoglobin','lowest_monocytes','male',
        'infection_present']

# EDA

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
admissions = pd.read_excel('aml_data_8.1.xlsx')
#admissions.head()
len(admissions)

In [ ]:
pos_ads = admissions[admissions['infection_present'] == 1]
print('There are '+str(len(pos_ads))+' infection-positive admissions.')
print('There are '+str(len(pos_ads.MRN.unique()))+' patients that were infected.')
#pos_ads.head()
neg_ads = admissions[admissions['infection_present'] == 0]
print('There are '+str(len(neg_ads))+' infection-negative admissions.')
#neg_ads.head()

# number of admissions
tot_ads = admissions.pivot_table(index='MRN',values='age',aggfunc='count')
tot_ads.plot.hist(bins=5).set_title('Amount of Patient Admissions')
plt.xlabel('Admissions Count')
plt.ylabel('Number of Patients')

In [ ]:
# num infections vs age range, pos & neg
sns.boxplot(x='infection_present', y='age', data= admissions).set_title('Age Distribution of Pos/Neg Infections')

In [ ]:
ad_race = admissions.pivot_table(index='race_code', values='MRN', aggfunc='count')
pos_race = pos_ads.pivot_table(index='race_code', values='MRN', aggfunc='count')
neg_race = neg_ads.pivot_table(index='race_code', values='MRN', aggfunc='count')
pos_pie=pos_race.plot.pie(y='MRN',figsize=(10,6)).set_title('Positive Infections by Race',fontsize=18)
neg_race.plot.pie(y='MRN',figsize=(10,6)).set_title('Negative Infections by Race',fontsize=18)

In [ ]:
ad_gender = admissions.pivot_table(index='gender', values='MRN', aggfunc='count')
pos_gender= pos_ads.pivot_table(index='gender', values='MRN', aggfunc='count')
pos_gender.plot.pie(y='MRN',figsize=(10,4)).set_title('Positive Infections by Gender',fontsize=18)
neg_gender= neg_ads.pivot_table(index='gender', values='MRN', aggfunc='count')
neg_gender.plot.pie(y='MRN',figsize=(10,4)).set_title('Negative Infections by Gender',fontsize=18)

In [ ]:
sns.boxplot(data=admissions, x='infection_present', y='age', hue='gender').set_title("Positive and Negative Cases Age Range by Gender");

In [ ]:
cond_plot = sns.FacetGrid(data=admissions, col='race_code', col_wrap=4)
cond_plot.fig.subplots_adjust(top=0.9) 
cond_plot.fig.suptitle("Positive and Negative Cases Age Range by Race")
cond_plot.map(sns.boxplot, 'infection_present', 'age')

In [ ]:
sns.boxplot(x= admissions['infection_present'], y=admissions['first_bmi_kg_m2']).set_title("BMI for Pos/Neg Infections");

In [ ]:
sns.scatterplot(data=admissions, x='age', y='first_bmi_kg_m2', hue='infection_present').set_title("Age and BMI for Positive/Negative Infections");

In [ ]:
sns.boxplot(data=admissions, x='infection_present', y='first_bmi_kg_m2', hue='gender').set_title("Positive and Negative Cases BMI Range by Gender");

In [ ]:
# los vs infection
sns.boxplot(x='infection_present', y='LOS', data= admissions).set_title('Length of Stay (days) of Pos/Neg Infections')

In [ ]:
# how many total icu admissions were there?
len(admissions)-sum(admissions.icu_los.isna())

In [ ]:
ads_icu = admissions.pivot_table(index='infection_present', values='icu_los',aggfunc='count')
pos_ratio = ads_icu.loc[1]/len(pos_ads)
neg_ratio = ads_icu.loc[0]/len(neg_ads)
ratio = [neg_ratio,pos_ratio]
ratio = pd.DataFrame(ratio)
ratio.plot.bar(y='icu_los').set_title('ICU Admissions Ratios of Pos/Neg Infections')
plt.xticks(rotation=0)

In [ ]:
# deaths
ads_dod = admissions.pivot_table(index='infection_present', values='dod',aggfunc='count')
pos_ratio2 = ads_dod.loc[1]/len(pos_ads)
neg_ratio2 = ads_dod.loc[0]/len(neg_ads)
ratio2 = [neg_ratio2,pos_ratio2]
ratio2 = pd.DataFrame(ratio2)
ratio2.plot.bar(y='dod').set_title('Deaths of Positive/Negative Infections')
plt.xticks(rotation=0)

In [ ]:
# distribution of infections over time
# look at infections per admissions (ratio) each year
pos_ads['infection_present'].groupby(pos_ads['admit_date'].dt.year).count().plot(kind="bar",figsize=(10,4)).set_title('Infections Over Time')
plt.xticks(rotation=0);

In [ ]:
# infection relapses
relapses = admissions.pivot_table(index='MRN',values='infection_present',aggfunc='sum')
relapses.plot.hist().set_title('Number of Infections Per Patient')
plt.xlabel('Number of Infections')
plt.ylabel('Number of Patients')

In [ ]:
# lowest_neutrophil
sns.boxplot(x='infection_present', y='lowest_neutrophil', data= admissions).set_title('Lowest Neutrophil Count of Admissions for Pos/Neg Infections')

In [ ]:
#CV_line_name
ad_cv = admissions.pivot_table(index='CV_buckets', values='MRN', aggfunc='count')
ad_cv.plot.pie(y='MRN',figsize=(10,8)).set_title('CV Lines per Admission',fontsize=18)